<a href="https://colab.research.google.com/github/cesar-ramosh/Gobernanzas-de-datos/blob/main/Sector_Precios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gobernanza de datos - Sector Precios

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Sector Precios/

Mounted at /content/drive
/content/drive/MyDrive/Sector Precios


In [ ]:
%cd /content/drive/MyDrive/Sector Precios/

/content/drive/MyDrive/Sector Precios


# Importación de datos

In [ ]:
# Cargar la segunda hoja en un DataFrame de Pandas y utilizar la primera columna como índice de fechas
# 398 productos
IPC_Productos = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Bolivia_Productos')
#IPC_Productos = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Bolivia_Productos', index_col=0)
# 13 variables divisiones
IPC_Division = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Divisiones')
#IPC_Division = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Divisiones', index_col=0)
# 9 departamentos + Nacional
IPC_Ciudad = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Ciudades')
# 3 variables
IPC_Alimento = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Alimentos')
#IPC_Alimento = pd.read_excel('01. Base Sector Precios.xlsx', sheet_name='Alimentos', index_col=0)

## Función para el cálculo de la inflación mensual, a 12 meses y acumulada

In [ ]:
import pandas as pd

def calcular_inflacion(df, columna_variable, columna_fecha):
    # Hacer una copia del DataFrame para evitar modificar el original
    df = df.copy()

    # Calcular la tasa de crecimiento mensual
    df[f'imes_{columna_variable}'] = df[columna_variable].pct_change() * 100

    # Calcular la tasa de crecimiento a 12 meses
    df[f'i12m_{columna_variable}'] = df[columna_variable].pct_change(periods=12) * 100

    # Convertir la columna de fecha a formato "datetime" si no está en ese formato
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])

    # Extraer el año de la columna de fecha
    df['year'] = df[columna_fecha].dt.year

    # Crear la función para el año de referencia
    min_year = df['year'].min()
    max_year = df['year'].max()

    for year in range(min_year, max_year):
        base_col = f"base_{year}"
        df[base_col] = df.loc[(df['year'] == year) & (df[columna_fecha].dt.month == 12), columna_variable].values[0]

    # Calcular la tasa de crecimiento acumulada utilizando el enfoque de referencia anual
    for year in range(min_year + 1, max_year + 1):
        prev_year = year - 1
        df.loc[df['year'] == year, f'icum_{columna_variable}'] = ((df[columna_variable] / df[f"base_{prev_year}"]) - 1) * 100

    # Eliminar las variables auxiliares
    df = df.drop(['year'] + [f"base_{year}" for year in range(min_year, max_year)], axis=1)

    return df

# Índice Precios al Consumidor por departamento

In [ ]:
IPC_Ciudad

,mes,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando
0,1993-01-01,26.163202,NaN,28.874885,21.911197,NaN,NaN,NaN,25.606815,NaN,NaN
1,1993-02-01,26.412003,NaN,29.372096,22.088338,NaN,NaN,NaN,25.646317,NaN,NaN
2,1993-03-01,26.398434,NaN,29.279760,22.077894,NaN,NaN,NaN,25.758710,NaN,NaN
3,1993-04-01,26.429421,NaN,29.179465,22.111790,NaN,NaN,NaN,25.951515,NaN,NaN
4,1993-05-01,26.630310,NaN,29.555209,22.270766,NaN,NaN,NaN,25.963425,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
364,2023-05-01,109.439785,107.720465,111.728514,110.378259,107.848700,107.953979,104.945418,108.027563,105.406641,108.659407
365,2023-06-01,109.678594,108.083533,111.932513,110.541804,108.210645,108.125089,105.150957,108.315414,105.450615,109.058022
366,2023-07-01,110.081702,109.083580,112.380421,111.154581,108.726100,109.165700,106.098239,108.424972,105.426013,109.145296
367,2023-08-01,110.506839,109.103324,113.280829,111.653623,109.172088,109.657499,105.959000,108.485973,105.740102,109.052151


## Limpieza de datos

In [ ]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
IPC_Ciudad = IPC_Ciudad.dropna(axis=0)
IPC_Ciudad.head(10)

,mes,Bolivia,Sucre,La Paz,Cochabamba,Oruro,Potosí,Tarija,Santa Cruz,Beni,Pando
182,2008-03-01,63.842495,64.415399,63.251928,60.001040,65.619475,66.618980,52.699891,66.402018,75.115948,78.054333
183,2008-04-01,64.313862,64.894046,63.646462,60.662801,65.824157,67.043223,52.961798,66.851485,75.645198,79.410778
184,2008-05-01,65.516355,65.941273,64.579039,61.512019,67.582691,69.451327,53.746839,68.441253,76.768770,80.625414
185,2008-06-01,66.341615,67.967391,65.647938,62.092102,68.938614,70.491526,54.604455,69.068068,77.173387,81.390116
186,2008-07-01,66.645505,68.095464,66.013454,62.348477,69.264672,70.461071,54.883008,69.401673,77.312484,81.722457
187,2008-08-01,67.080994,68.319670,66.547968,62.675628,68.867301,70.993449,55.720747,69.845060,77.562671,82.302577
188,2008-09-01,67.671950,68.129845,67.045705,63.709219,69.611001,71.258465,56.856623,70.280782,77.918130,84.286108
189,2008-10-01,67.792055,68.593582,67.322245,63.661749,70.205332,71.629532,56.865641,70.302683,78.016591,84.675791
190,2008-11-01,67.881343,68.206260,67.532584,63.737581,70.203433,71.523136,57.792752,70.280782,77.946570,84.575131
191,2008-12-01,68.171467,67.882063,67.799330,64.021916,70.406692,71.233072,57.412722,70.772807,78.489685,84.094005


## Cálculo inflación IPC por ciudades

In [ ]:
Infla_bol = calcular_inflacion(IPC_Ciudad, 'Bolivia', 'mes')
Infla_bol = Infla_bol.iloc[:, [0, 11, 12, 13]]
Infla_scr = calcular_inflacion(IPC_Ciudad, 'Sucre', 'mes')
Infla_scr = Infla_scr.iloc[:, [11, 12, 13]]
Infla_lpz = calcular_inflacion(IPC_Ciudad, 'La Paz', 'mes')
Infla_lpz = Infla_lpz.iloc[:, [11, 12, 13]]
Infla_cba = calcular_inflacion(IPC_Ciudad, 'Cochabamba', 'mes')
Infla_cba = Infla_cba.iloc[:, [11, 12, 13]]
Infla_oru = calcular_inflacion(IPC_Ciudad, 'Oruro', 'mes')
Infla_oru = Infla_oru.iloc[:, [11, 12, 13]]
Infla_pot = calcular_inflacion(IPC_Ciudad, 'Potosí', 'mes')
Infla_pot = Infla_pot.iloc[:, [11, 12, 13]]
Infla_tar = calcular_inflacion(IPC_Ciudad, 'Tarija', 'mes')
Infla_tar = Infla_tar.iloc[:, [11, 12, 13]]
Infla_scz = calcular_inflacion(IPC_Ciudad, 'Santa Cruz', 'mes')
Infla_scz = Infla_scz.iloc[:, [11, 12, 13]]
Infla_ben = calcular_inflacion(IPC_Ciudad, 'Beni', 'mes')
Infla_ben = Infla_ben.iloc[:, [11, 12, 13]]
Infla_pdo = calcular_inflacion(IPC_Ciudad, 'Pando', 'mes')
Infla_pdo = Infla_pdo.iloc[:, [11, 12, 13]]
# Concatenar los DataFrames en uno solo
infla_depto = pd.concat([Infla_bol, Infla_scr, Infla_lpz, Infla_cba, Infla_oru, Infla_pot, Infla_tar, Infla_scz, Infla_ben, Infla_pdo], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
infla_depto = infla_depto.dropna(axis=0)
# Formato de fechas
infla_depto['mes'] = infla_depto['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_depto = infla_depto.round(4)

# Índice Precios al Consumidor por división

In [ ]:
IPC_Division

,mes,Bolivia,Alimentos y bebidas no alcohólicas,Bebidas alcohólicas y tabaco,Prendas de vestir y calzado,Vivienda y servicios básicos,"Muebles, bienes y servicios domésticos",Salud,Transporte,Comunicaciones,Recreación y cultura,Educación,Alimentos y bebidas consumidos fuera del hogar,Bienes y servicios diversos
0,1992-03-01,24.551016,20.787881,NaN,35.001393,26.626877,23.769239,20.629507,NaN,NaN,31.287264,25.521613,NaN,31.305815
1,1992-04-01,24.615680,20.773083,NaN,35.110960,27.021873,23.855080,20.909262,NaN,NaN,31.496949,25.783236,NaN,31.641182
2,1992-05-01,24.786690,21.030243,NaN,35.226133,26.929463,23.889374,20.889832,NaN,NaN,32.575250,25.838054,NaN,31.781612
3,1992-06-01,24.947639,21.118356,NaN,35.421447,27.447277,24.219344,20.991879,NaN,NaN,32.801991,25.975349,NaN,32.197117
4,1992-07-01,25.135306,21.221324,NaN,35.637529,27.988685,24.414355,21.008381,NaN,NaN,33.420825,26.066485,NaN,32.698548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2023-05-01,109.439785,111.898267,101.252511,98.382092,107.422718,110.951771,117.916596,108.334825,98.136610,105.048489,129.611842,113.707557,104.053272
375,2023-06-01,109.678594,112.268197,101.304292,98.530254,107.532931,111.835975,118.395626,108.454371,97.999285,104.842433,129.667319,113.894058,104.423866
376,2023-07-01,110.081702,112.843084,101.720439,98.711938,107.671401,111.672831,118.599029,110.313604,97.898623,105.153536,129.535703,114.061162,104.694549
377,2023-08-01,110.506839,114.236076,101.525841,98.589244,107.737937,111.764493,118.680517,110.516130,97.716401,105.101224,129.679026,114.231051,104.830152


## Limpieza de datos

In [ ]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
IPC_Division = IPC_Division.dropna(axis=0)
IPC_Division.head(10)

,mes,Bolivia,Alimentos y bebidas no alcohólicas,Bebidas alcohólicas y tabaco,Prendas de vestir y calzado,Vivienda y servicios básicos,"Muebles, bienes y servicios domésticos",Salud,Transporte,Comunicaciones,Recreación y cultura,Educación,Alimentos y bebidas consumidos fuera del hogar,Bienes y servicios diversos
192,2008-03-01,63.842495,56.615130,49.093485,74.082568,68.692183,59.933426,56.796616,74.419416,100.336748,84.959796,68.235514,56.900409,67.841085
193,2008-04-01,64.313862,57.078113,49.831134,74.452012,69.468718,60.799488,57.285302,74.425607,100.315732,85.138690,68.028202,57.655139,68.402421
194,2008-05-01,65.516355,59.898042,51.193838,74.951647,69.817732,61.318871,57.825194,74.340357,100.173132,84.595075,67.896197,58.646365,69.151059
195,2008-06-01,66.341615,61.269321,51.624286,75.412427,70.191131,62.041387,58.127352,74.427934,100.037820,85.004534,67.989071,59.698707,70.319263
196,2008-07-01,66.645505,61.246696,52.590698,75.960821,70.231749,62.781696,58.624710,74.702472,99.390160,85.028364,68.069530,60.629766,71.195858
197,2008-08-01,67.080994,61.760375,52.873168,76.384313,70.178676,63.765683,58.679244,74.587884,98.536705,85.053481,68.398579,61.608615,72.051372
198,2008-09-01,67.671950,62.737005,53.073190,76.542532,70.027626,64.418711,58.650083,74.719516,98.499918,85.191348,68.388156,62.919140,72.522180
199,2008-10-01,67.792055,62.449463,53.904592,76.730223,70.098489,64.865818,59.598296,74.687944,98.507287,85.074207,68.372559,63.738173,72.967202
200,2008-11-01,67.881343,62.029056,54.911059,76.801436,70.313479,65.723566,60.137032,75.194307,98.135868,85.231695,68.400253,64.104438,73.208116
201,2008-12-01,68.171467,62.271857,55.475135,77.344790,70.540754,66.170164,60.726743,75.537602,97.753165,85.456627,68.416197,64.542434,73.338289


## Cálculo inflación por división

In [ ]:
Infla_bol = calcular_inflacion(IPC_Division, 'Bolivia', 'mes')
Infla_bol = Infla_bol.iloc[:, [0, 14, 15, 16]]
Infla_abnh = calcular_inflacion(IPC_Division, 'Alimentos y bebidas no alcohólicas', 'mes')
Infla_abnh = Infla_abnh.iloc[:, [14, 15, 16]]
Infla_bayt = calcular_inflacion(IPC_Division, 'Bebidas alcohólicas y tabaco', 'mes')
Infla_bayt = Infla_bayt.iloc[:, [14, 15, 16]]
Infla_pvyc = calcular_inflacion(IPC_Division, 'Prendas de vestir y calzado', 'mes')
Infla_pvyc = Infla_pvyc.iloc[:, [14, 15, 16]]
Infla_vysb = calcular_inflacion(IPC_Division, 'Vivienda y servicios básicos', 'mes')
Infla_vysb = Infla_vysb.iloc[:, [14, 15, 16]]
Infla_mbsd = calcular_inflacion(IPC_Division, 'Muebles, bienes y servicios domésticos', 'mes')
Infla_mbsd = Infla_mbsd.iloc[:, [14, 15, 16]]
Infla_salu = calcular_inflacion(IPC_Division, 'Salud', 'mes')
Infla_salu = Infla_salu.iloc[:, [14, 15, 16]]
Infla_tran = calcular_inflacion(IPC_Division, 'Transporte', 'mes')
Infla_tran = Infla_tran.iloc[:, [14, 15, 16]]
Infla_comu = calcular_inflacion(IPC_Division, 'Comunicaciones', 'mes')
Infla_comu = Infla_comu.iloc[:, [14, 15, 16]]
Infla_reyc = calcular_inflacion(IPC_Division, 'Recreación y cultura', 'mes')
Infla_reyc = Infla_reyc.iloc[:, [14, 15, 16]]
Infla_educ = calcular_inflacion(IPC_Division, 'Educación', 'mes')
Infla_educ = Infla_educ.iloc[:, [14, 15, 16]]
Infla_acfh = calcular_inflacion(IPC_Division, 'Alimentos y bebidas consumidos fuera del hogar', 'mes')
Infla_acfh = Infla_acfh.iloc[:, [14, 15, 16]]
Infla_bysd = calcular_inflacion(IPC_Division, 'Bienes y servicios diversos', 'mes')
Infla_bysd = Infla_bysd.iloc[:, [14, 15, 16]]
# Concatenar los DataFrames en uno solo
infla_divis = pd.concat([Infla_bol, Infla_abnh, Infla_bayt, Infla_pvyc, Infla_vysb, Infla_mbsd, Infla_salu, Infla_tran, Infla_comu, Infla_reyc, Infla_educ, Infla_acfh, Infla_bysd], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
infla_divis = infla_divis.dropna(axis=0)
# Formato de fechas
#infla_divis['mes'] = infla_divis['mes'].dt.strftime('%b-%y')
infla_divis.loc[:, 'mes'] = infla_divis['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_divis = infla_divis.round(4)
#infla_divis = infla_divis.round(2)

# Índice Precios al Consumidor - Alimentos

In [ ]:
IPC_Alimento

,mes,IPC general,Alimentos,No alimentos
0,1992-03-01,24.551019,20.766725,28.431095
1,1992-04-01,24.615683,20.751941,28.600619
2,1992-05-01,24.786632,21.008840,28.645760
3,1992-06-01,24.947641,21.096863,28.897687
4,1992-07-01,25.135309,21.199726,29.191124
...,...,...,...,...
374,2023-05-01,109.439785,112.241117,107.419029
375,2023-06-01,109.678594,112.543204,107.612191
376,2023-07-01,110.081702,112.978151,107.992333
377,2023-08-01,110.506839,113.928148,108.038858


## Limpieza de datos

In [ ]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
IPC_Alimento = IPC_Alimento.dropna(axis=0)
IPC_Alimento.head(10)

,mes,IPC general,Alimentos,No alimentos
0,1992-03-01,24.551019,20.766725,28.431095
1,1992-04-01,24.615683,20.751941,28.600619
2,1992-05-01,24.786632,21.008840,28.645760
3,1992-06-01,24.947641,21.096863,28.897687
4,1992-07-01,25.135309,21.199726,29.191124
5,1992-08-01,25.366300,21.498953,29.317180
6,1992-09-01,25.382414,21.394178,29.497120
7,1992-10-01,25.513882,21.532326,29.612665
8,1992-11-01,25.640874,21.596088,29.819191
9,1992-12-01,25.725535,21.584962,30.029928


## Cálculo inflación Alimentos

In [ ]:
Infla_gral = calcular_inflacion(IPC_Alimento, 'IPC general', 'mes')
Infla_gral = Infla_gral.iloc[:, [0, 4, 5, 6]]
Infla_alim = calcular_inflacion(IPC_Alimento, 'Alimentos', 'mes')
Infla_alim = Infla_alim.iloc[:, [4, 5, 6]]
Infla_nalim = calcular_inflacion(IPC_Alimento, 'No alimentos', 'mes')
Infla_nalim = Infla_nalim.iloc[:, [4, 5, 6]]

# Concatenar los DataFrames en uno solo
infla_alimen = pd.concat([Infla_gral, Infla_alim, Infla_nalim], axis=1)
import pandas as pd
# Eliminar observaciones que contienen valores nulos
infla_alimen = infla_alimen.dropna(axis=0)
# Formato de fechas
infla_alimen.loc[:, 'mes'] = infla_alimen['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_alimen = infla_alimen.round(4)

# Índice de precios al consumidor por productos (397 productos)

In [ ]:
IPC_Productos

,mes,IPC general,Pan corriente,Pan especial,Galletas,Panetón,Torta,Empanada al horno,Queques,Llauchas,...,Servicio de guarderías,Seguro relacionado con la salud,Seguro relacionado con el transporte,Servicios de asesoramiento jurídico,Gastos en pompas fúnebres y en otros servicios de funerales,Pago por fotocopias y otras reproducciones de documentos,Emisión de certificados,Roseta de inspección vehicular,Cédula de identidad CI,Servicios religiosos
0,2017-12-01,100.860240,1.944171,0.245399,0.505634,0.093038,0.202355,0.191127,0.043872,0.027014,...,0.111559,0.070172,0.089397,0.050412,0.039202,0.065664,0.040209,0.013366,0.016013,0.000761
1,2018-01-01,101.167616,1.944034,0.244824,0.509821,0.093293,0.201589,0.192480,0.043617,0.026634,...,0.113687,0.070172,0.090389,0.049549,0.039969,0.064478,0.040181,0.013366,0.016013,0.000761
2,2018-02-01,101.490505,1.943415,0.245220,0.510412,0.093293,0.202737,0.192860,0.043703,0.026966,...,0.115020,0.070172,0.090389,0.048790,0.039992,0.064465,0.040209,0.013366,0.016013,0.000761
3,2018-03-01,101.360929,1.943415,0.245769,0.510882,0.093293,0.201695,0.194118,0.043529,0.027181,...,0.114775,0.070146,0.089751,0.053675,0.039751,0.064700,0.040209,0.013366,0.016013,0.000761
4,2018-04-01,101.216604,1.943438,0.245780,0.510664,0.093293,0.204056,0.194084,0.043794,0.026900,...,0.115309,0.069293,0.088896,0.051172,0.040464,0.064665,0.040559,0.013366,0.016013,0.000761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2023-05-01,109.439785,1.953798,0.256659,0.607927,0.103828,0.228242,0.213634,0.052812,0.034141,...,0.127588,0.071047,0.091391,0.059424,0.049589,0.069964,0.040209,0.013366,0.016013,0.001016
66,2023-06-01,109.678594,1.953798,0.256739,0.614528,0.103828,0.228875,0.215229,0.053132,0.034141,...,0.126776,0.070876,0.091391,0.058934,0.049869,0.070031,0.040209,0.013366,0.016013,0.001016
67,2023-07-01,110.081702,1.953798,0.256776,0.620104,0.103828,0.228739,0.214718,0.052785,0.034141,...,0.126693,0.071047,0.089642,0.059833,0.049571,0.069432,0.040209,0.013366,0.016013,0.001016
68,2023-08-01,110.506839,1.953798,0.256680,0.624646,0.103828,0.229961,0.215214,0.053006,0.034336,...,0.125834,0.071047,0.089642,0.059944,0.050283,0.069342,0.040209,0.013366,0.016013,0.001016


## Limpieza de datos

In [ ]:
import pandas as pd
# Eliminar observaciones que contienen valores nulos
IPC_Productos = IPC_Productos.dropna(axis=0)
IPC_Productos.head(10)

,mes,IPC general,Pan corriente,Pan especial,Galletas,Panetón,Torta,Empanada al horno,Queques,Llauchas,...,Servicio de guarderías,Seguro relacionado con la salud,Seguro relacionado con el transporte,Servicios de asesoramiento jurídico,Gastos en pompas fúnebres y en otros servicios de funerales,Pago por fotocopias y otras reproducciones de documentos,Emisión de certificados,Roseta de inspección vehicular,Cédula de identidad CI,Servicios religiosos
0,2017-12-01,100.860240,1.944171,0.245399,0.505634,0.093038,0.202355,0.191127,0.043872,0.027014,...,0.111559,0.070172,0.089397,0.050412,0.039202,0.065664,0.040209,0.013366,0.016013,0.000761
1,2018-01-01,101.167616,1.944034,0.244824,0.509821,0.093293,0.201589,0.192480,0.043617,0.026634,...,0.113687,0.070172,0.090389,0.049549,0.039969,0.064478,0.040181,0.013366,0.016013,0.000761
2,2018-02-01,101.490505,1.943415,0.245220,0.510412,0.093293,0.202737,0.192860,0.043703,0.026966,...,0.115020,0.070172,0.090389,0.048790,0.039992,0.064465,0.040209,0.013366,0.016013,0.000761
3,2018-03-01,101.360929,1.943415,0.245769,0.510882,0.093293,0.201695,0.194118,0.043529,0.027181,...,0.114775,0.070146,0.089751,0.053675,0.039751,0.064700,0.040209,0.013366,0.016013,0.000761
4,2018-04-01,101.216604,1.943438,0.245780,0.510664,0.093293,0.204056,0.194084,0.043794,0.026900,...,0.115309,0.069293,0.088896,0.051172,0.040464,0.064665,0.040559,0.013366,0.016013,0.000761
5,2018-05-01,101.335284,1.943437,0.246044,0.510735,0.093293,0.204428,0.194324,0.044077,0.027055,...,0.115300,0.070153,0.089043,0.051233,0.040475,0.064597,0.040209,0.013366,0.016013,0.000761
6,2018-06-01,101.454934,1.943437,0.246221,0.509447,0.093293,0.204228,0.194741,0.044482,0.027266,...,0.115206,0.070153,0.089021,0.050088,0.041340,0.064523,0.040209,0.013366,0.016013,0.000761
7,2018-07-01,101.497113,1.943437,0.246260,0.509464,0.093293,0.204280,0.195097,0.045022,0.027123,...,0.114985,0.070153,0.088932,0.050826,0.041079,0.066319,0.040209,0.013366,0.016013,0.000761
8,2018-08-01,101.671782,1.943437,0.246365,0.510523,0.093293,0.203773,0.194931,0.045363,0.027121,...,0.115446,0.070153,0.088916,0.050111,0.041303,0.066430,0.040209,0.013366,0.016013,0.000761
9,2018-09-01,101.584122,1.943437,0.246592,0.512121,0.093293,0.204569,0.194634,0.045808,0.027121,...,0.115586,0.070153,0.089135,0.051216,0.041734,0.066430,0.040209,0.013366,0.016013,0.000761


## Cálculo de la inflación de los 397 productos

In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado IPC_Productos con 398 variables.
variables = IPC_Productos.columns.tolist()
variables = [var for var in variables if var != "mes"]

infla_resultados = pd.DataFrame()  # Dataframe para almacenar los resultados

for variable in variables:
    infla = calcular_inflacion(IPC_Productos, variable, 'mes')
    infla = infla[['mes', f'imes_{variable}', f'i12m_{variable}', f'icum_{variable}']]

    # Combina los resultados en un DataFrame más grande
    infla_resultados = pd.concat([infla_resultados, infla], axis=1)

# Ahora, infla_resultados contendrá las columnas de inflación para todas las variables
import pandas as pd
# Eliminar observaciones que contienen valores nulos
infla_produc = infla_resultados.dropna(axis=0)
# Formato de fechas
# Eliminar la variable repetida
infla_produc = infla_produc.loc[:, ~infla_produc.columns.duplicated(keep='first')]
infla_produc.loc[:, 'mes'] = infla_produc['mes'].dt.strftime('%b-%y')
# Redondear los decimales
infla_produc = infla_produc.round(4)


# Exportar resultados a un archivo excel

In [ ]:
# Supongamos que tienes un DataFrame llamado df y deseas guardarlo en un archivo llamado 'mi_dataframe.xlsx' en tu carpeta de Google Drive.
ruta_archivo = '/content/drive/MyDrive/Sector Precios/Resultados Sector Precios.xlsx'
infla_depto.to_excel(ruta_archivo, sheet_name="inflación departamento", index=False)  # El argumento index=False evita que se guarde el índice en el archivo Excel.

# Carga el archivo Excel existente.
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    infla_divis.to_excel(writer, sheet_name="inflación divisiones", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    infla_alimen.to_excel(writer, sheet_name="inflación alimentos", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    infla_produc.to_excel(writer, sheet_name="inflación productos", index=False)


# Organización dataframes

In [ ]:
#===============================================#
# Seleccionamos las lista de todas las variables de departamentos.
variables_depto = infla_depto.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
depto_imes = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('imes_')] ]
depto_imes = depto_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
depto_i12m = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('i12m_')] ]
depto_i12m = depto_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
depto_icum = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('icum_')] ]
depto_icum = depto_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#
# Seleccionamos las lista de todas las variables de divisiones.
variables_divis = infla_divis.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
divis_imes = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('imes_')] ]
divis_imes = divis_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
divis_i12m = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('i12m_')] ]
divis_i12m = divis_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
divis_icum = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('icum_')] ]
divis_icum = divis_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#
# Seleccionamos las lista de todas las variables de productos.
variables_produc = infla_produc.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
produc_imes = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('imes_')] ]
produc_imes = produc_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
produc_i12m = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('i12m_')] ]
produc_i12m = produc_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
produc_icum = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('icum_')] ]
produc_icum = produc_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#
# Seleccionamos las lista de todas las variables de alimentos.
variables_alimen = infla_alimen.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
alimen_imes = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('imes_')] ]
alimen_imes = alimen_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
alimen_i12m = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('i12m_')] ]
alimen_i12m = alimen_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
alimen_icum = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('icum_')] ]
alimen_icum = alimen_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#

## Exportar todos los dataframes por categoria
- Departamento
- Divisiones
- Alimentos
- Productos

In [ ]:
# Supongamos que tienes un DataFrame llamado df y deseas guardarlo en un archivo llamado 'mi_dataframe.xlsx' en tu carpeta de Google Drive.
# Inflación departamental
ruta_archivo = '/content/drive/MyDrive/Sector Precios/Resultados Precios Desagregado.xlsx'
depto_imes.to_excel(ruta_archivo, sheet_name="inf. mens. depto", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    depto_i12m.to_excel(writer, sheet_name="inf. a12m. depto", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    depto_icum.to_excel(writer, sheet_name="inf. acum. depto", index=False)
# Inflación por Divisiones
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_imes.to_excel(writer, sheet_name="inf. mens. división", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_i12m.to_excel(writer, sheet_name="inf. a12m. división", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    divis_icum.to_excel(writer, sheet_name="inf. acum. división", index=False)
# Inflación por Productos (397)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    produc_imes.to_excel(writer, sheet_name="inf. mens. producto", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    produc_i12m.to_excel(writer, sheet_name="inf. a12m. producto", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    produc_icum.to_excel(writer, sheet_name="inf. acum. producto", index=False)
# Inflación por Alimentos
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_imes.to_excel(writer, sheet_name="inf. mens. alim.", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_i12m.to_excel(writer, sheet_name="inf. a12m. alim.", index=False)
with pd.ExcelWriter(ruta_archivo, mode='a', engine='openpyxl') as writer:
    alimen_icum.to_excel(writer, sheet_name="inf. acum. alim.", index=False)

In [ ]:
#===============================================#
# Seleccionamos las lista de todas las variables de departamentos.
variables_depto = infla_depto.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
depto_imes = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('imes_')] ]
depto_imes = depto_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
depto_i12m = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('i12m_')] ]
depto_i12m = depto_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
depto_icum = infla_depto[ ['mes'] + [variable for variable in infla_depto.columns if variable.startswith('icum_')] ]
depto_icum = depto_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#
# Seleccionamos las lista de todas las variables de divisiones.
variables_divis = infla_divis.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
divis_imes = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('imes_')] ]
divis_imes = divis_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
divis_i12m = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('i12m_')] ]
divis_i12m = divis_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
divis_icum = infla_divis[ ['mes'] + [variable for variable in infla_divis.columns if variable.startswith('icum_')] ]
divis_icum = divis_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#
# Seleccionamos las lista de todas las variables de productos.
variables_produc = infla_produc.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
produc_imes = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('imes_')] ]
produc_imes = produc_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
produc_i12m = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('i12m_')] ]
produc_i12m = produc_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
produc_icum = infla_produc[ ['mes'] + [variable for variable in infla_produc.columns if variable.startswith('icum_')] ]
produc_icum = produc_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#
# Seleccionamos las lista de todas las variables de alimentos.
variables_alimen = infla_alimen.columns.tolist()
# Creamos un dataframe con todas las inflaciones mensual/12meses/acumulada por departamento
alimen_imes = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('imes_')] ]
alimen_imes = alimen_imes.rename(columns=lambda x: x.replace('imes_', '') if x.startswith('imes_') else x) # Retornamos nombre original
alimen_i12m = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('i12m_')] ]
alimen_i12m = alimen_i12m.rename(columns=lambda x: x.replace('i12m_', '') if x.startswith('i12m_') else x) # Retornamos nombre original
alimen_icum = infla_alimen[ ['mes'] + [variable for variable in infla_alimen.columns if variable.startswith('icum_')] ]
alimen_icum = alimen_icum.rename(columns=lambda x: x.replace('icum_', '') if x.startswith('icum_') else x) # Retornamos nombre original
#===============================================#

In [ ]:
import pandas as pd
import numpy as np
import datetime

# Simular datos para df1
date_rng = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
df1_data = {
    'mes': date_rng,
    'Variable1_df1': np.random.rand(len(date_rng)),
    'Variable2_df1': np.random.rand(len(date_rng)),
    'Variable3_df1': np.random.rand(len(date_rng))
}
df1 = pd.DataFrame(df1_data)

# Simular datos para df2
df2_data = {
    'mes': date_rng,
    'Variable1_df2': np.random.rand(len(date_rng)),
    'Variable2_df2': np.random.rand(len(date_rng)),
    'Variable3_df2': np.random.rand(len(date_rng)),
    'Variable4_df2': np.random.rand(len(date_rng))
}
df2 = pd.DataFrame(df2_data)

# Simular datos para df3
date_rng = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
df3_data = {
    'mes': date_rng,
    'Variable1_df3': np.random.rand(len(date_rng)),
    'Variable2_df3': np.random.rand(len(date_rng)),
    'Variable3_df3': np.random.rand(len(date_rng)),
    'Variable4_df3': np.random.rand(len(date_rng)),
    'Variable5_df3': np.random.rand(len(date_rng))
}
df3 = pd.DataFrame(df3_data)

# Ahora puedes utilizar df1, df2 y df3 con datos simulados en el código anterior para crear el gráfico.
####
#####
########
#########
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Resto del código (creación de fig1, fig2, fig3 y personalización de las subfiguras)...

# Crear una figura con tres subfiguras
fig = make_subplots(rows=1, cols=3)

# Agregar las subfiguras al gráfico
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=1, col=3)

# Personalizar el diseño de la figura
fig.update_layout(title='Gráfico con Subfiguras')
fig.update_xaxes(title_text='Fecha', row=1, col=1)
fig.update_xaxes(title_text='Fecha', row=1, col=2)
fig.update_xaxes(title_text='Fecha', row=1, col=3)

# Mostrar el gráfico interactivo
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'infla_depto' contiene los datos de inflación
fig = px.area(depto_imes, x="mes", y=depto_imes.columns[1:], title="Inflación mensual por Departamento")

# Agregar un slider para la fecha "mes"
fig.update_xaxes(rangeslider_visible=True)


import pandas as pd
import plotly.express as px

# Asumiendo que 'depto_imes' contiene los datos de inflación
fig = px.area(depto_imes, x="mes", y=depto_imes.columns[1:], title="Inflación mensual por Departamento")

# Calcular el número de observaciones que deseas mostrar
num_observaciones = 60

# Calcular la fecha de inicio basada en el número de observaciones
fecha_inicio = depto_imes['mes'].max() - pd.DateOffset(months=num_observaciones)

# Agregar un slider para la fecha "mes" con el rango de las últimas 60 observaciones
fig.update_xaxes(rangeslider_visible=True, range_x=[fecha_inicio, depto_imes['mes'].max()])


# Personalizar etiquetas y leyenda, fondo blanco y tamaño de letra a 18
fig.update_layout(
    yaxis_title="En porcentaje",
    xaxis_title="Período",
    legend_title="Departamento",
    plot_bgcolor='white',  # Fondo blanco
    font=dict(family="LaTeX", size=18),  # Tipo de fuente en formato LaTeX y tamaño 18
    height=900,  # Ajustar la altura de la figura
    width=1200,   # Ajustar el ancho de la figura
    barmode='group'  # Configurar el modo de barras para que no sean acumuladas
)

# Mostrar el gráfico
fig.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Custom TB Handler failed, unregistering
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-146-13f43ee4d1a9>", line 21, in <cell line: 21>
    fecha_inicio = depto_imes['mes'].max() - pd.DateOffset(months=num_observaciones)
TypeError: unsupported operand type(s) for -: 'str' and 'DateOffset'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 281, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes

# Gráficas

In [ ]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'infla_depto' contiene los datos de inflación
fig = px.area(infla_depto, x="mes", y=infla_depto.columns[1:], title="Inflación por Departamento")

# Agregar un slider para la fecha "mes"
fig.update_xaxes(rangeslider_visible=True)

# Personalizar etiquetas y leyenda, fondo blanco y tamaño de letra a 18
fig.update_layout(
    yaxis_title="En porcentaje",
    xaxis_title="Período",
    legend_title="Departamento",
    plot_bgcolor='white',  # Fondo blanco
    font=dict(family="LaTeX", size=18),  # Tipo de fuente en formato LaTeX y tamaño 18
    height=900,  # Ajustar la altura de la figura
    width=1200,   # Ajustar el ancho de la figura
    barmode='group'  # Configurar el modo de barras para que no sean acumuladas
)

# Mostrar el gráfico
fig.show()

In [ ]:
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_mapbox(df, geojson=geojson, color="Bergeron",
                           locations="district", featureidkey="properties.district",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)
fig.show()

In [ ]:
!pip install -U plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 44.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [ ]:
!pip install Folium

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Load your DataFrame
df = IPC_Ciudad

# Create a figure
fig = go.Figure()

# Add traces for each variable
fig.add_trace(go.Scatter(x=df['mes'], y=df['Inflacion_Mensual_Bolivia'], mode='lines', name='Inflacion Mensual'))
fig.add_trace(go.Scatter(x=df['mes'], y=df['Inflacion_12Meses_Bolivia'], mode='lines', name='Inflacion 12 Meses'))
fig.add_trace(go.Scatter(x=df['mes'], y=df['Inflacion_Acumulada_Bolivia'], mode='lines', name='Inflacion Acumulada'))

# Set title
fig.update_layout(title_text="Inflación en Bolivia a lo largo del tiempo")

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()


In [ ]:
IPC_Ciudad.columns

Index(['mes', 'Bolivia', 'Sucre', 'Conurbación La Paz',
       'Región Metropolitana de Kanata', 'Oruro', 'Potosí', 'Tarija',
       'Conurbación Santa Cruz', 'Trinidad', 'Cobija',
       'Inflacion_Mensual_Bolivia', 'Inflacion_12Meses_Bolivia', 'year',
       'base_2008', 'base_2009', 'base_2010', 'base_2011', 'base_2012',
       'base_2013', 'base_2014', 'base_2015', 'base_2016', 'base_2017',
       'base_2018', 'base_2019', 'base_2020', 'base_2021', 'base_2022',
       'Inflacion_Acumulada_Bolivia', 'Inflacion_Mensual_Sucre',
       'Inflacion_12Meses_Sucre', 'Inflacion_Acumulada_Sucre',
       'Inflacion_Mensual_Conurbación La Paz',
       'Inflacion_12Meses_Conurbación La Paz',
       'Inflacion_Acumulada_Conurbación La Paz',
       'Inflacion_Mensual_Región Metropolitana de Kanata',
       'Inflacion_12Meses_Región Metropolitana de Kanata',
       'Inflacion_Acumulada_Región Metropolitana de Kanata',
       'Inflacion_Mensual_Oruro', 'Inflacion_12Meses_Oruro',
       'Inflacion_A

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Load your DataFrame
df = IPC_Ciudad

# Create a figure
fig = go.Figure()

# Add bar traces for each variable
fig.add_trace(go.Bar(x=df['mes'], y=df['Inflacion_Mensual_Bolivia'], name='Inflacion Mensual', marker_color='blue'))
fig.add_trace(go.Bar(x=df['mes'], y=df['Inflacion_12Meses_Bolivia'], name='Inflacion 12 Meses', marker_color='green'))
fig.add_trace(go.Bar(x=df['mes'], y=df['Inflacion_Acumulada_Bolivia'], name='Inflacion Acumulada', marker_color='red'))

# Set title
fig.update_layout(title_text="Inflación en Bolivia a lo largo del tiempo")

# Set white background
fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()



In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

# Crear datos simulados
np.random.seed(42)
n_points = 50
x = pd.date_range(start='2023-01-01', periods=n_points, freq='D')
y = np.cumsum(np.random.randn(n_points))

# Crear un DataFrame con los datos
df = pd.DataFrame({'Fecha': x, 'Valor': y})

# Crear el gráfico dinámico con Plotly
fig = px.line(df, x='Fecha', y='Valor', title='Gráfico Dinámico con Plotly', labels={'Fecha': 'Fecha', 'Valor': 'Valor'})
fig.show()
